In [2]:
import numpy as np
import pandas as pd

import os

import warnings
warnings.filterwarnings('ignore')

path = '/opt/ml/input/data/train/'

train = pd.read_csv(path + 'train_ratings.csv')

year_data = pd.read_csv(os.path.join(path, 'years.tsv'), sep='\t')
writer_data = pd.read_csv(os.path.join(path, 'writers.tsv'), sep='\t')
title_data = pd.read_csv(os.path.join(path, 'titles.tsv'), sep='\t')
genre_data = pd.read_csv(os.path.join(path, 'genres.tsv'), sep='\t')
director_data = pd.read_csv(os.path.join(path, 'directors.tsv'), sep='\t')

In [3]:
# 모델 시작 전 인코딩하는 과정입니다.
user2idx = {user:idx for idx, user in enumerate(train['user'].unique())}
idx2user = {idx:user for idx, user in enumerate(train['user'].unique())}

## 패딩 때문에 0 비워놓음
item2idx = {item:(idx+1) for idx, item in enumerate(train['item'].unique())}
idx2item = {(idx+1):item for idx, item in enumerate(train['item'].unique())}

genre2idx = {genre:(idx+1) for idx, genre in enumerate(genre_data['genre'].unique())}
idx2genre = {(idx+1):genre for idx, genre in enumerate(genre_data['genre'].unique())}

train_new = train.copy()
genre_data_new = genre_data.copy()

train_new['user'] = train_new['user'].map(user2idx)
train_new['item'] = train_new['item'].map(item2idx)

genre_data_new['item'] = genre_data_new['item'].map(item2idx)
genre_data_new['genre'] = genre_data_new['genre'].map(genre2idx)
train_new.to_csv(os.path.join(path, 'train_new.csv'), index = False)
genre_data_new.to_csv(os.path.join(path, 'genre_data_new.csv'), index = False)

In [ ]:
# 모델 학습 후 디코딩하는 과정입니다.
out_path = './output'
output = pd.read_csv(os.path.join(out_path, 'submission.csv'))
output['user'] = output['user'].map(idx2user)
output['item'] = output['item'].map(idx2item)
output.to_csv(os.path.join(out_path, 'deco_submission.csv'), index = False)